# Project: Data Modeling with Apache Cassandra

<img src="./images/cassandralogo.png" width="250" height="250">

# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

### The code works with the Generated CSV file titled <font color=blue>event_datafile_new.csv</font>, located within the Workspace directory. The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

### Creating a connection to the database

In [5]:
from cassandra.cluster import Cluster
try:
    # Make a connection to a Cassandra instance on local machine (127.0.0.1)
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

### Creating a keyspace

In [6]:
# Create a Keyspace named sparkify
try:
    session.execute("""
                        CREATE KEYSPACE IF NOT EXISTS sparkify 
                        WITH REPLICATION = 
                        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                    """)
except Exception as e:
    print(e)

#### Connectng to our Keyspace.

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. With Apache Cassandra we model the database tables on the queries we want to run.

### We want to ask following first question of our data:

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Data Modeling for first query

##### Since data is looking for `sessionId` and `itemInSession` we will make the primary key comprised of both the `sessionId` and `itemInSession` as composite partition key and in order to uniquely identify each row we will add the `userID` as clustering column 

In [8]:
query_create_table_songinfo = """CREATE TABLE IF NOT EXISTS song_info_in_session (
                                            session_id int, 
                                            item_in_session int, 
                                            artist text, 
                                            song_title text, 
                                            song_length float, 
                                            user_id int,
                                            PRIMARY KEY ((session_id, item_in_session), user_id)
                                            )"""

try:
    session.execute(query_create_table_songinfo)
except Exception as e:
    print(e)                    

#### Insert the data into the table

In [9]:
file = 'event_datafile_new.csv'

query = """INSERT INTO song_info_in_session (
                                                session_id, 
                                                item_in_session, 
                                                artist, 
                                                song_title, 
                                                song_length, 
                                                user_id
                                            ) """
query = query + "VALUES (%s, %s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]), int(line[10])))   
        except Exception as e:
            print(e)

#### Validate that we have correctly inserted the data by counting the total number of records inserted

In [10]:
query_select_song_info = """SELECT count(*)
                            FROM song_info_in_session"""

try:
    rows = session.execute(query_select_song_info)
    print (rows[0].count)
except Exception as e:
    print(e)

6820


#### Performing a SELECT to verify that the data have been inserted into each table

In [11]:
## SELECT statement verifies the data was entered into the table
query_select_song_info = """SELECT *
                            FROM song_info_in_session
                            WHERE session_id=338 AND item_in_session=4"""

try:
    rows = session.execute(query_select_song_info)
    for row in rows:
        print (row.session_id, row.item_in_session, row.artist, 
               row.artist,  row.song_title, row.song_length, row.user_id)
except Exception as e:
    print(e)

338 4 Faithless Faithless Music Matters (Mark Knight Dub) 495.30731201171875 50


### We want to ask following second question of our data:

#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Data Modeling for second query

##### Since data is looking for `userid` and `sessionid` we will make the primary key comprised of both the `userid` and `sessionid` as composite partition key and in order to uniquely identify each row and sort by `itemInSession`, `firstName` and `lastName` we will add the `itemInSession`, `firstName` and `lastName` as clustering column 

In [12]:
query_create_user_song_preferences = """CREATE TABLE IF NOT EXISTS user_song_preferenes (
                                        artist text,
                                        song_title text,
                                        user_id int,
                                        first_name text,
                                        last_name text,
                                        item_in_session int, 
                                        session_id int,
                                        PRIMARY KEY ((user_id, session_id), item_in_session, first_name, last_name)
                                        )"""
              
try:
    session.execute(query_create_user_song_preferences)
except Exception as e:
    print(e) 

#### Insert the data into the table

In [13]:
file = 'event_datafile_new.csv'

query_insert_user_song_preferenes = """INSERT INTO user_song_preferenes (
                                                artist, 
                                                song_title, 
                                                user_id, 
                                                first_name, 
                                                last_name, 
                                                item_in_session,
                                                session_id
                                            ) """
query_insert_user_song_preferenes = query_insert_user_song_preferenes + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(query_insert_user_song_preferenes, 
                            (line[0], line[9], int(line[10]), line[1], line[4], int(line[3]), int(line[8])))  
        except Exception as e:
            print(e)

#### Performing a SELECT to verify that the data have been inserted into each table

In [14]:
## The SELECT statement verifies the data was entered into the table
query_select_user_song_preferences = """SELECT *
                            FROM user_song_preferenes
                            WHERE user_id=10 AND session_id=182"""

try:
    rows = session.execute(query_select_user_song_preferences)
    for row in rows:
        print (row.artist, row.song_title, row.user_id, row.first_name, 
               row.last_name, row.item_in_session, row.session_id)
except Exception as e:
    print(e)

Down To The Bone Keep On Keepin' On 10 Sylvie Cruz 0 182
Three Drives Greece 2000 10 Sylvie Cruz 1 182
Sebastien Tellier Kilometer 10 Sylvie Cruz 2 182
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 10 Sylvie Cruz 3 182


#### Validate that we have correctly inserted the data by counting the total number of records inserted

In [15]:
query_select_user_song_preferenes = """SELECT count(*)
                            FROM user_song_preferenes"""

try:
    rows = session.execute(query_select_user_song_preferenes)
    print (rows[0].count)
except Exception as e:
    print(e)

6820


### We want to ask following third question of our data:

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Data Modeling for third query

##### Since data is looking for `song` we will make the primary key comprised of both the `song` as simple partition key and in order to uniquely identify each row and sort by `firstName`, `lastName`, `sessionid`, `userid`, `itemInSession` we will add the `firstName`, `lastName`, `sessionid`, `userid`, `itemInSession` as clustering column 

In [16]:
query_create_user_music_history = """CREATE TABLE IF NOT EXISTS user_music_history (
                                         user_id int,
                                         first_name text,
                                         last_name text,
                                         song text, 
                                         item_in_session int,
                                         session_id int,
                                         PRIMARY KEY (song, first_name, last_name, session_id, user_id, item_in_session)
                                    )"""

try:
    session.execute(query_create_user_music_history)
except Exception as e:
    print(e) 

#### Insert the data into the table

In [17]:
file = 'event_datafile_new.csv'

query_insert_user_music_history = """INSERT INTO user_music_history (
                                                user_id, 
                                                first_name, 
                                                last_name, 
                                                song,
                                                item_in_session,
                                                session_id
                                            ) """
query_insert_user_music_history = query_insert_user_music_history + "VALUES (%s, %s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(query_insert_user_music_history, 
                            (int(line[10]), line[1], line[4], line[9], int(line[3]), int(line[8])))  
        except Exception as e:
            print(e)

#### Performing a SELECT to verify that the data have been inserted into each table

In [18]:
## The SELECT statement verifies the data was entered into the table
query_select_user_music_history = """SELECT *
                            FROM user_music_history
                            WHERE song='All Hands Against His Own'"""

try:
    rows = session.execute(query_select_user_music_history)
    for row in rows:
        print (row.user_id, row.first_name, row.last_name, row.song, row.item_in_session, row.session_id)
except Exception as e:
    print (e)

29 Jacqueline Lynch All Hands Against His Own 50 559
95 Sara Johnson All Hands Against His Own 31 152
80 Tegan Levine All Hands Against His Own 25 611


#### Validate that we have correctly inserted the data by counting the total number of records inserted

In [19]:
query_select_user_music_history = """SELECT count(*)
                            FROM user_music_history"""

try:
    rows = session.execute(query_select_user_music_history)
    print (rows[0].count)
except Exception as e:
    print(e)

6820


### Drop the tables before closing out the sessions

In [20]:
## Drop the table before closing out the sessions
query_drop_table_song_info_in_session = "drop table song_info_in_session"
try:
    rows = session.execute(query_drop_table_song_info_in_session)
except Exception as e:
    print(e)

In [21]:
query_drop_table_user_song_preferenes = "drop table user_song_preferenes"
try:
    rows = session.execute(query_drop_table_user_song_preferenes)
except Exception as e:
    print(e)

In [22]:
query_drop_table_user_music_history = "drop table user_music_history"
try:
    rows = session.execute(query_drop_table_user_music_history)
except Exception as e:
    print(e)

In [23]:
query = "drop keyspace sparkify"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()